In [ ]:
# Installs the latest version of the Qiskit meta-package for circuit creation.
!pip install qiskit -U


In [ ]:
!pip install qiskit-ibm-runtime -U

In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime import Session, Options
from qiskit.visualization import plot_histogram
import numpy as np
from collections import Counter

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_cloud", token="", instance="")


In [34]:
# Define the quantum circuit for Swap Test
def create_swap_test_circuit():
    qc = QuantumCircuit(3, 1)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.h(0)
    qc.measure(0, 0)
    return qc

# Define function to run the quantum circuit using SamplerV2
def run_swap_test_circuit(circuit):
    backend = FakeManilaV2()  # Replace with your actual backend from IBM Quantum
    transpiled_circuit = transpile(circuit, backend=backend)

    # Initialize the SamplerV2
    sampler = SamplerV2(backend=backend)

   # Run the circuit using the SamplerV2
    shots = 1024
    job = sampler.run([transpiled_circuit], shots=shots)
    result = job.result()

 # Extract and process counts from the result object
    counts = extract_counts_from_result(result)

    return counts

def extract_counts_from_result(result):
    """Extract and process counts from the result object."""
    try:
        # Result contains SamplerPubResult, extract the BitArray
        pub_result = result[0]
        data_bin = pub_result.data
        bit_array = data_bin.c

        # Convert BitArray to a numpy array or list
        if hasattr(bit_array, 'to_list'):
            bit_array_list = bit_array.to_list()
        else:
            bit_array_list = list(bit_array)

        # Convert list of bitstrings to counts
        counts = {}
        for bits in bit_array_list:
            bitstring = ''.join(map(str, bits))
            counts[bitstring] = counts.get(bitstring, 0) + 1

        return counts

    except Exception as e:
        print(f"Error extracting counts from result: {e}")
        return {}



In [35]:
# Sample function to generate synthetic data for KNN classification
def generate_synthetic_data():
    # Generate synthetic features and labels
    X = np.random.rand(100, 2)  # 100 samples, 2 features
    y = np.random.randint(0, 2, 100)  # Binary classification
    return X, y

# Train and evaluate a KNN classifier
def train_and_evaluate_knn(X_train, y_train, X_test, y_test):
    # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=5)

    # Train the classifier
    knn.fit(X_train, y_train)

    # Make predictions
    y_pred = knn.predict(X_test)

    # Evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

print("Circuit before training:")
print(circuit.draw())

# Create and run the Swap Test circuit
circuit = create_swap_test_circuit()
counts = run_swap_test_circuit(circuit)

# Print the swap test results
print("Swap Test Measurement Results:")
print(counts)



# Generate synthetic data for demonstration
X, y = generate_synthetic_data()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train and evaluate the KNN classifier
accuracy = train_and_evaluate_knn(X_train, y_train, X_test, y_test)
print(f"KNN Classifier Accuracy: {accuracy:.2f}")

# Print the circuit after applying the swap test
print("\nCircuit after swap test:")
print(circuit.draw())

Circuit before training:
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├──■────■──┤ H ├┤M├
     └───┘┌─┴─┐  │  └───┘└╥┘
q_1: ─────┤ X ├──┼────────╫─
          └───┘┌─┴─┐      ║ 
q_2: ──────────┤ X ├──────╫─
               └───┘      ║ 
c: 1/═════════════════════╩═
                          0 
Error extracting counts from result: The input array must have at least two axes.
Swap Test Measurement Results:
{}
KNN Classifier Accuracy: 0.47

Circuit after swap test:
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├──■────■──┤ H ├┤M├
     └───┘┌─┴─┐  │  └───┘└╥┘
q_1: ─────┤ X ├──┼────────╫─
          └───┘┌─┴─┐      ║ 
q_2: ──────────┤ X ├──────╫─
               └───┘      ║ 
c: 1/═════════════════════╩═
                          0 


<ipython-input-34-cc4ffc600dc8>:17: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = SamplerV2(backend=backend)
/usr/local/lib/python3.10/dist-packages/qiskit_ibm_runtime/fake_provider/fake_backend.py:468: RuntimeWarning: Aer not found, using qiskit.BasicSimulator and no noise.
  warnings.warn(
